In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
x = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        x += 1 
    

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [3]:
sam = pd.read_csv("/kaggle/input/2023-irtmhw3/hw3_sam.csv")
train = pd.DataFrame(np.loadtxt("/kaggle/input/2023-irtmhw3/training.txt", dtype = int))

In [4]:
par_path = "/kaggle/input/newsdoc/IRTM"
allc = []
training = []
for i in range(13):
    cla = []
    for j in range(1,16):
        sub = str(train.iat[i,j]) + ".txt"
        training.append(train.iat[i,j])

In [5]:
def extract_doc(d_idx): # doc index
    par_path = "/kaggle/input/newsdoc/IRTM"
    file_path = os.path.join(par_path, str(d_idx)+".txt")
    d = [] 
    if os.path.isfile(file_path) and sub.endswith(".txt"):
        with open(file_path, 'r') as f:
            content = f.read()
            d.append(content)
            
    doc = []
    tmp = []
    # split
    for con in d:
        con = con.split()
        tmp.extend(con)
        
    tmpd = {}
    for term in tmp:
        new_t = term.replace('!', '').replace('(', '').replace(')', '').replace('-', '').replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace(';', '').replace(':', '').replace('"', '').replace('\'', '').replace(',', '').replace('<', '').replace('>', '').replace('.', '').replace('/', '').replace('?', '').replace('@', '').replace('#', '').replace('$', '').replace('%', '').replace('^', '').replace('&', '').replace('*', '').replace('*', '').replace('_', '').replace('~', '').replace('`', '').replace(' ', '')
        if any(char.isnumeric() for char in new_t): # remove numeric
            continue
        if len(new_t) > 0:
            new_t = new_t.lower() # lower
            new_t = ps.stem(new_t) # stemming
            tmpd[new_t] = None # remove duplicate
        doc = list(tmpd)
        
    with open('/kaggle/input/stopwords/stopwords.txt') as st:
        stopword = list(st)
        
    stopwords = [words.split() for words in stopword]
    # put every stopword in different list into the same list
    stopwords = sum(stopwords, []) 
    
    fdoc = []
    for term in doc:
        if term not in stopwords:
            fdoc.append(term)
    
    return fdoc

In [6]:
dataset = []

current = []
for idx, doc_idx in enumerate(training):
    current.append(extract_doc(doc_idx))
    
    if(idx+1)%15 == 0:
        dataset.append(current)
        current = []
        
if current:
    dataset.append(current)


In [7]:
voc = []

for cla in dataset:
    for doc in cla:
        for term in doc:
            if term not in voc:
                voc.append(term)


data = []
for cla in dataset:
    c = []
    for term in cla:
        if term not in c:
            c.extend(term)
    data.append(c)

### M*|C| P(X=t|c) parameters

In [8]:
# term present on / present off / absent on / absent off topic

df = pd.DataFrame(None, columns = voc, index = range(13))
for term in voc:
    for cidx, topic in enumerate(dataset):
        n11 = sum(term in doc for doc in topic)
        n10 = 15 - n11
        n01 = sum(term in odoc for ocla in dataset[:cidx]+dataset[cidx+1:] for odoc in ocla)
        n00 = 180 - n01 # (13-1) * 15 = 180

        n = [
            [n11, n10],
            [n01, n00]
        ]
        df[term][cidx] = n

In [9]:
def binom_exp(n):
    n11 = n[0][0]
    n01 = n[1][0]
    n10 = n[0][1]
    n00 = n[1][1] 
    N = 195 # # of all training dataset
    pt = (n11+n01) / N
    p1 = n11 / (n11+n10)
    p2 = n01 / (n01+n00)
    
    LH1 = (pt**n11) * ((1-pt)**n10)* (pt**n01) * ((1-pt)**n00)
    LH2 = (p1**n11) * ((1-p1)**n10)* (p2**n01) * ((1-p2)**n00)
    
    LLR = -2*math.log(LH1/LH2)
    return LLR

In [10]:
LLR_df = pd.DataFrame(None, columns = voc, index = range(13))

for term in voc:
    for i in range(13):
        LLR_df[term][i] = binom_exp(df[term][i])

In [11]:
tmpdf = LLR_df.sum().sort_values(ascending = False)
tmpdf = tmpdf.head(500)
term500 = list(tmpdf.index)

In [12]:
LLR_500 = LLR_df[term500]
df500 = df[term500]

In [13]:
def TrainMNB(df):
    N = 195 # # training dataset
    
    cla_len = []
    prior = []
    for c in range(df.shape[0]):
        tmp = df.iloc[c, :]
        tmp = sum([i[0][0] for i in tmp])
        cla_len.append(tmp)
        
    condprob = pd.DataFrame(None, columns = term500, index = range(13))
    
    for c in range(df.shape[0]):
        Nc = 15 # # doc in class
        prior.append(Nc / N)
        V = len(term500)
        for i, t in enumerate(term500):
            Tct = df[t][c][0][0]
            condprob[t][c] = (Tct+1) / (cla_len[c]+V)
    return prior, condprob

In [14]:
prior, condprob = TrainMNB(df500)

In [15]:
def ApplyMNB(prior, condprob, d_idx):
    w = extract_doc(d_idx)
    score = []
    for c in range(df.shape[0]):
        score.append(math.log(prior[c]))
        for t in w:
            if t not in term500:
                continue
            score[c] += math.log(condprob[t][c])
    return np.argmax(score)+1

In [16]:
testing = []
for i in range(1,1096):
    if i not in training:
        testing.append(i)

In [17]:
result = {}
for i in testing:
    c = ApplyMNB(prior, condprob, i)
    result[i] = c

In [18]:
ori = pd.read_csv("/kaggle/input/2023-irtmhw3/hw3_sam.csv", delimiter=',')
ori['Value'] = ori['Id'].map(result)
ori.to_csv("/kaggle/working/hw3_res.csv", index=False)

In [19]:
fin = pd.read_csv("/kaggle/working/hw3_res.csv", delimiter=',')

In [20]:
fin

,Id,Value
0,17,2
1,18,2
2,20,2
3,21,2
4,22,2
...,...,...
895,1091,6
896,1092,6
897,1093,6
898,1094,6
